# AI agents with LangChain

In [12]:
import os
from dotenv import load_dotenv, find_dotenv
import os
import getpass
from typing import Annotated, Any, Dict, List, Optional
from dataclasses import dataclass, field

# Imports LangChain / LangGraph
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command, Send
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, ConfigDict, Field, PrivateAttr
from dotenv import load_dotenv
import asyncio
import uuid


load_dotenv(find_dotenv())

True

### Agents sur langchain avec create_react_agent

In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent

@tool
def get_word_length(word: str) -> int:
    """Retourne la longueur (nombre de lettres) d'un mot donné."""
    return len(word)

@tool
def magic_multiplier(a: int, b: int) -> int:
    """Multiplie deux nombres d'une manière magique."""
    return a * b

tools = [get_word_length, magic_multiplier]
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent_graph = create_agent(model, tools)

print("--- Démarrage de l'Agent Moderne (LangGraph) ---")

query = "Quelle est la longueur du mot 'Anticonstitutionnellement' multipliée par 5 ?"

inputs = {"messages": [("user", query)]}

result = agent_graph.invoke(inputs)

last_message = result["messages"][-1]
print(last_message.content)

print("\n--- Historique des interactions ---")
for msg in result["messages"]:
    print(f"[{msg.type.upper()}]: {msg.content}")

--- Démarrage de l'Agent Moderne (LangGraph) ---
La longueur du mot 'Anticonstitutionnellement' multipliée par 5 est 125.

--- Historique des interactions ---
[HUMAN]: Quelle est la longueur du mot 'Anticonstitutionnellement' multipliée par 5 ?
[AI]: 
[TOOL]: 25
[AI]: 
[TOOL]: 125
[AI]: La longueur du mot 'Anticonstitutionnellement' multipliée par 5 est 125.


### Multi-agentic systems with LangGraph - 

In [14]:
import os
import getpass
import asyncio
from typing import List, Any, Optional
from dataclasses import dataclass, field
import uuid

# --- Imports LangChain / LangGraph ---
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

# --- Le "New Stuff" ---
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

from pydantic import BaseModel, ConfigDict, PrivateAttr
from dotenv import load_dotenv



@dataclass
class EmbeddingModelWrapper:
    embeddings: OpenAIEmbeddings = field(default_factory=OpenAIEmbeddings)

model_wrapper = EmbeddingModelWrapper()
recall_vector_store = InMemoryVectorStore(model_wrapper.embeddings)

@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    document = Document(page_content=memory, id=str(uuid.uuid4()))
    recall_vector_store.add_documents([document])
    return "Memory saved."

@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""
    documents = recall_vector_store.similarity_search(query, k=3)
    return [document.page_content for document in documents]

@tool
def tell_joke(topic: str) -> str:
    """Tells a joke about a specific topic."""
    return f"Why did the {topic} cross the road? To get to the other side! (Ha ha)"

@tool
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers together."""
    return a + b


class HighLevelSupervisorAgent(BaseModel):
    model: ChatOpenAI
    
    # Le graphe compilé sera stocké ici
    _app: Any = PrivateAttr(default=None)

    model_config = ConfigDict(
        extra='forbid',
        arbitrary_types_allowed=True, 
    )

    def model_post_init(self, __context: Any) -> None:
        """
        Construit l'architecture Supervisor en utilisant la librairie 'langgraph_supervisor'.
        """

        # A. Création des Agents Spécialistes (Workers)
        # ---------------------------------------------
        # On utilise create_react_agent standard.
        # Note: On donne un nom explicite à chaque agent, c'est ce nom que le superviseur utilisera.
        
        joke_agent = create_react_agent(
            model=self.model,
            tools=[tell_joke],
            name="joke_expert", # Le nom interne de l'agent
            prompt="You are a funny assistant. Your goal is to tell jokes when asked."
        )

        math_agent = create_react_agent(
            model=self.model,
            tools=[add_numbers],
            name="math_expert",
            prompt="You are a math assistant. You only do additions."
        )

        # B. Création du Superviseur
        # ---------------------------------------------------
        # 'create_supervisor' fait tout le travail difficile pour nous :
        # 1. Il crée automatiquement les outils de handoff pour "joke_expert" et "math_expert".
        # 2. Il gère le graphe d'état et le routage.
        # 3. Il ajoute un noeud spécial pour que le superviseur puisse répondre directement.
        
        workflow = create_supervisor(
            agents=[joke_agent, math_agent], # Liste des workers
            tools=[save_recall_memory, search_recall_memories], # Outils propres au superviseur
            model=self.model,
            prompt=(
                "You are a supervisor managing two assistants: 'joke_expert' and 'math_expert'. "
                "Delegate tasks to them. If the task is finished or general, answer yourself."
            )
        )

        self._app = workflow.compile()

    def get_graph(self):
        return self._app

    def draw_image(self):
        try:
            with open("high_level_supervisor.png", "wb") as f:
                f.write(self._app.get_graph().draw_mermaid_png())
            print("Graph saved to high_level_supervisor.png")
        except Exception as e:
            print(f"Could not draw graph: {e}")

    async def arun(self, user_input: str):
        print(f"\nUser Input: {user_input}")
        
        initial_state = {"messages": [{"role": "user", "content": user_input}]}
        
        async for chunk in self._app.astream(initial_state, subgraphs=True):
            if "next" in chunk:
                print(f"🔄 Routing to: {chunk['next']}")
            elif "messages" in chunk:
                pass 
        final_state = await self._app.ainvoke(initial_state)
        print(f"🤖 Final Answer: {final_state['messages'][-1].content}")

    def run(self, user_input: str):
        """Wrapper intelligent pour gérer Jupyter vs Terminal"""
        try:
            loop = asyncio.get_running_loop()
        except RuntimeError:
            loop = None

        if loop and loop.is_running():
            print("⚠️ Dans Jupyter, utilisez 'await agent.arun(...)'")
            return self.arun(user_input)
        else:
            asyncio.run(self.arun(user_input))


if __name__ == "__main__":
    if not os.environ.get("OPENAI_API_KEY"):
        print("Please set OPENAI_API_KEY")
        # exit(1)

    agent = HighLevelSupervisorAgent(
        model=ChatOpenAI(model="gpt-4o", temperature=0),
    )

    agent.draw_image()
    
    print("--- TEST 1: MATH ---")
    if asyncio.get_event_loop().is_running():
        await agent.arun("Calculate 50 + 50")
    else:
        agent.run("Calculate 50 + 50")
    
    print("\n--- TEST 2: JOKE ---")
    if asyncio.get_event_loop().is_running():
        await agent.arun("Tell me a joke about Python code.")

Graph saved to high_level_supervisor.png
--- TEST 1: MATH ---

User Input: Calculate 50 + 50


/tmp/ipykernel_2977499/1156794806.py:77: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  joke_agent = create_react_agent(
/tmp/ipykernel_2977499/1156794806.py:84: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  math_agent = create_react_agent(


🤖 Final Answer: The result of 50 + 50 is 100.

--- TEST 2: JOKE ---

User Input: Tell me a joke about Python code.
🤖 Final Answer: I hope you enjoyed the joke! If you have any more requests, feel free to ask.
